# Combinando Classificadores
#### Como combinar classificadores para analisar a predição de um algoritmo.

### Usando mais de um classificador para fazer predições

In [1]:
# função que solicita o diretório do 'dataframe'
def solicita_diretorio():
    
    # solicita ao usuário o diretório onde o arquivo está localizado
    diretorio = str(input('Informe o diretorio do arquivo CSV: '))
    
    # quebra de linha
    print('\n')
    
    # retorna uma string com a localização do arquivo
    return diretorio
# fim da função 'solicita_diretorio'

In [2]:
# função que realiza o pré-processamento dos dados
def pre_processa():
    
    # importando a biblioteca pandas do python
    import pandas as pd
    
    # importa a função que recebe a localização do arquivo CSV
    localiza_csv = solicita_diretorio()
    
    # importando a base de dados para o objeto "dataframe"
    dataframe = pd.read_csv(localiza_csv, encoding = 'utf-8', sep = ',')

    # substituindo valores inconsistentes no campo idades pela média das idades consistentes
    dataframe.loc[dataframe.age < 0, 'age'] = 40.92

    # separando os atributos previsores do meta classe
    previsores = dataframe.iloc[:, 1:4].values
    classe = dataframe.iloc[:, 4].values

    # importando a biblioteca sklearn do python
    from sklearn.impute import SimpleImputer
    # função "SimpleImputer" responsável por corrigir valores faltantes na base de dados

    # importando a biblioteca numpy do python
    import numpy as np

    # criando o objeto "imputer"
    imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")

    # fazendo o treinamento com a base de dados para correção de valores faltantes
    imputer = imputer.fit(previsores[:,0:3])

    # corrigindo os valores faltantes usando o objeto criado "imputer"
    previsores[:, 0:3] = imputer.transform(previsores[:, 0:3])

    # importando a biblioteca sklearn do python
    from sklearn.preprocessing import StandardScaler
    # a função "StandardScaler" tem a função de escalonas toda a base de dados para corrigir a discrepância
    # dos valores

    # criando o objeto "scaler"
    scaler = StandardScaler()

    # escalonando todos os atributos para auxiliar no cálculo de distâncias euclidianas
    previsores = scaler.fit_transform(previsores)
    
    # função retorna os atributos previsores e meta
    return previsores, classe
# fim da função 'pre-processa'

In [3]:
# criando uma função para padronizar os atributos
def setup_registro(renda, idade, emprestimo):
    
    # importando a biblioteca numpy do python
    import numpy as np
    
    # criando uma lista com os valores do registro
    registro = [[renda, idade, emprestimo]]
    
    # transformando a lista em um numpy array
    registro = np.asarray(registro)
    
    # tranformando em um vetor coluna para realizar o escalonamento dos atributos
    registro = registro.reshape(-1, 1)
    
    # importando a biblioteca sklearn do python
    from sklearn.preprocessing import StandardScaler
    
    # criando o objeto 'scaler'
    scaler = StandardScaler()
    
    # aplicando o escalonamento nos atributos
    registro = scaler.fit_transform(registro)
    
    # retransformando o vetor coluna em um vetor linha novamente
    registro = registro.reshape(-1, 3)
    
    # função retorna um array numpy escalonado
    return registro
# fim da função 'setup_registro'

In [4]:
# função criada para receber novos registros a serem analisados
def recebe_dados():
    
    # recebendo a renda do usuário
    renda = float(input('Informe a renda do usuário: '))
    # recebendo a idade do usuário
    idade = float(input('Informe a idade do usuário: '))
    # recebendo a quantia de empréstimo solicitado pelo usuário
    emprestimo = float(input('Informe o empréstimo solicitado pelo usuário: '))
    
    # aplica a padronização dos atributos nos dados recebidos
    dados = setup_registro(renda, idade, emprestimo)
    
    # função retorna um array numpy escalonado
    return dados
# fim da função 'recebe_dados'

In [5]:
# função que mostra na saída de dados os resultados majoritários dos classificadores
def analisa_dados(pagador, nao_pagador):
    
    # quebra de linha
    print('\n')
    
    # se os resultados do algoritmo indicarem classe '0' maior que classe '1'
    if pagador > nao_pagador:
        print('Cliente pagará o empréstimo')
    # se os resultados do algoritmo indicarem classe '0' igual a classe '1'
    elif pagador == nao_pagador:
        print('Cliente Duvidoso')
    # se os resultados do algoritmo indicarem classe '1' maior que classe '0'
    else:
        print('Cliente não pagará o empréstimo')
    
    # função não retorna nenhum valor
    return None
# fim da função analisa dados

In [6]:
# função que obtém os resultados dos classificadores para cada classe
def mescla_classificadores(nb, cnb, ad, cad, rf, crf, kn, ckn, rl, crl, sv, csv, rd, crd):
    
    # criando variáveis para armazenar os resultados majoritários dos classificadores
    paga = 0
    nao_paga = 0
    # definindo um limiar aceitável de acerto em 98.0%
    confianca_minima = 0.98
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if cnb >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Naive bayes
        if nb[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if cad >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Árvores de Decisão
        if ad[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if crf >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Random Forest
        if rf[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if ckn >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo KNN
        if kn[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if crl >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Regressão Logística
        if rl[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if csv >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Máquina de Vetores de Suporte
        if sv[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # se o resultado não for maior ou igual que o limiar ele é rejeitado
    if crd >= confianca_minima:
        # analisando o resultado previsto do registro para o algoritmo Redes Neurais
        if rd[0] == 0:
            paga += 1
        else:
            nao_paga += 1
    
    # faz a análise dos resultados obtidos com a função 'analisa_dados'
    analisa_dados(paga, nao_paga)
    
    # função não retorna nenhum valor
    return None
# fim da função 'mescla_classificadores'

In [7]:
def classificacao_IA():
    
    # importando a biblioteca pickle do python
    import pickle 
    
    # importando os classificadores para o código python

    naive_bayes = pickle.load(open('Classificadores Salvos/naive-bayes_finalizado.sav', 'rb'))
    arvore_decisao = pickle.load(open('Classificadores Salvos/arvore-decisao_finalizado.sav', 'rb'))
    random_forest = pickle.load(open('Classificadores Salvos/random-forest_finalizado.sav', 'rb'))
    knn = pickle.load(open('Classificadores Salvos/knn_finalizado.sav', 'rb'))
    regressao_logistica = pickle.load(open('Classificadores Salvos/regressao-logistica_finalizado.sav', 'rb'))
    svm = pickle.load(open('Classificadores Salvos/maquina-vetores-suporte_finalizado.sav', 'rb'))
    redes_neurais = pickle.load(open('Classificadores Salvos/redes-neurais_finalizado.sav', 'rb'))
    
    # obtendo os atributos previsores e os atributos meta
    previsores, classe = pre_processa()
    
    # visualizando a predição de acertos para cada uma das base de dados

    print('Resultado Naive Bayes: {:.2f}%'.format(naive_bayes.score(previsores, classe) * 100))
    print('Resultado Árvore de Decisão: {:.2f}%'.format(arvore_decisao.score(previsores, classe) * 100))
    print('Resultado Random Forest: {:.2f}%'.format(random_forest.score(previsores, classe) * 100))
    print('Resultado KNN: {:.2f}%'.format(knn.score(previsores, classe) * 100))
    print('Resultado Regressão Logística: {:.2f}%'.format(regressao_logistica.score(previsores, classe) * 100))
    print('Resultado Máquina de Vetores de Suporte: {:.2f}%'.format(svm.score(previsores, classe) * 100))
    print('Resultado Redes Neurais: {:.2f}%'.format(redes_neurais.score(previsores, classe) * 100))
    
    # quebra de linha
    print('\n')
    
    # recebendo do usuário os registros do cliente
    registro = recebe_dados()
    
    # obtendo a predição da classe em cada classificador criado usando o 'novo_registro'
    
    resp_naive_bayes = naive_bayes.predict(registro)
    resp_arvore_decisao = arvore_decisao.predict(registro)
    resp_random_forest = random_forest.predict(registro)
    resp_knn = knn.predict(registro)
    resp_regressao_logistica = regressao_logistica.predict(registro)
    resp_svm = svm.predict(registro)
    resp_mlp = redes_neurais.predict(registro)
    
    # obtendo a probabilidade de classificação para cada um dos atributos meta
    
    proba_naive_bayes = naive_bayes.predict_proba(registro)
    proba_arvore_decisao = arvore_decisao.predict_proba(registro)
    proba_random_forest = random_forest.predict_proba(registro)
    proba_knn = knn.predict_proba(registro)
    proba_regressao_logistica = regressao_logistica.predict_proba(registro)
    proba_svm = svm.predict_proba(registro)
    proba_mlp = redes_neurais.predict_proba(registro)
    
    # obtendo a confiança de predição para cada um dos algoritmos (método de limiar)
    
    conf_naive_bayes = proba_naive_bayes.max()
    conf_arvore_decisao = proba_arvore_decisao.max()
    conf_random_forest = proba_random_forest.max()
    conf_knn = proba_knn.max()
    conf_regressao_logistica = proba_regressao_logistica.max()
    conf_svm = proba_svm.max()
    conf_mlp = proba_mlp.max()
    
    # obtendo uma predição para o registro informado
    mescla_classificadores(resp_naive_bayes, conf_naive_bayes, resp_arvore_decisao, conf_arvore_decisao,
                           resp_random_forest, conf_random_forest, resp_knn, conf_knn, 
                           resp_regressao_logistica, conf_regressao_logistica, resp_svm, conf_svm,
                           resp_mlp, conf_mlp)
    
    # função não retorna nenhum valor
    return None
# fim da função 'classificador_IA'

In [8]:
# definindo a função principal 
def main():
    
    # chamando a função 'classificacao_IA' na função principal
    classificacao_IA()
    
    # função retorna um valor que indica que tudo ocorreu bem
    return None
# fim da função principal

In [10]:
# chamada da função principal
main()

Informe o diretorio do arquivo CSV:  Dados de Credito.csv




Resultado Naive Bayes: 92.60%
Resultado Árvore de Decisão: 100.00%
Resultado Random Forest: 100.00%
Resultado KNN: 99.20%
Resultado Regressão Logística: 94.80%
Resultado Máquina de Vetores de Suporte: 98.90%
Resultado Redes Neurais: 99.95%




Informe a renda do usuário:  7000
Informe a idade do usuário:  18
Informe o empréstimo solicitado pelo usuário:  10000




Cliente não pagará o empréstimo


### Alguma dúvida? Entre em contato comigo:

- [Me envie um e-mail](mailto:alysson.barbosa@ee.ufcg.edu.br);